In [16]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv
load_dotenv('sendgrid.env')
import os
import datetime
from datetime import date

In [18]:
def data_from_row(row):
    row_data = {}
    for cell in row.select('td'):
        row_data[cell['headers'][0]] = cell.text
    return row_data

def retrieve_page(page_index,params):
    url = 'https://ocrcas.ed.gov/open-investigations?' + '&'.join([k + '=' + str(v) for k, v in zip(params.keys(),params.values())]) + f'&page={page_index}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    return soup

def page_has_data(page):
    return len(page.select('table')) > 0

def scrape_page(page):
    table = page.select('table')[0].select('tbody')[0]
    df = pd.DataFrame([data_from_row(row) for row in table.select('tr')])
    return df

In [14]:
params = {
    'field_ois_state':'All',
    'field_ois_discrimination_statute':'All',
    'field_ois_type_of_discrimination':'All',
    'items_per_page':20,
    'field_ois_institution':'',
    'field_ois_institution_type':752,
    'field_open_investigation_date_1':str(date.today() - datetime.timedelta(days=7)),
    'field_open_investigation_date_2':str(date.today()),
    'field_open_investigation_date':'',
    'field_open_investigation_date_3':'',
}

In [19]:
tables = []
for i in range(0,99999999):
    page = retrieve_page(i,params)
    if page_has_data(page):
        tables.append(scrape_page(page))
    else:
        break
df = pd.concat(tables)

In [22]:
df.columns = ['STATE','INST','INST_TYPE','DISCRIMINATION_TYPE','INVEST_START_DATES']

In [25]:
df = df[['STATE','INST','DISCRIMINATION_TYPE','INVEST_START_DATES']]
for col in ['STATE','INST','DISCRIMINATION_TYPE']:
    df[col] = df[col].str.strip()

In [97]:
df.to_csv('../data/investigations_all.csv')

In [ ]:
# using SendGrid's Python Library
# https://github.com/sendgrid/sendgrid-python
import os
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail

message = Mail(
    from_email='declan@declanrjb.com',
    to_emails='dbradley@chronicle.com',
    subject=f'{len(df["INST"])} DoE investigations opened last week',
    html_content=f'<p>Investigations spanned {len(df["INST"].unique())} institutions in {len(df["STATE"].unique())} states.</p><br>{df.to_html()}')
try:
    sg = SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
    sg.send(message)
except Exception as e:
    print(e.message)